In [12]:
import torch
import torch.nn as nn

In [13]:
class NeuralNetwork(nn.Module):

  def __init__(self):
    super(NeuralNetwork,self).__init__()

    self.layer1 = nn.Sequential(
      nn.Conv2d(1,16,3,1,1),
      nn.BatchNorm2d(16),
      nn.ReLU(),
      nn.MaxPool2d(2,2)
    )

    self.layer2 = nn.Sequential(
      nn.Conv2d(16,32,3,1,1),
      nn.BatchNorm2d(32),
      nn.ReLU(),
      nn.MaxPool2d(2,2)
    )

    self.layer3 = nn.Sequential(
      nn.Conv2d(32,64,3,1,1),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.MaxPool2d(2,2)
    )

    self.layer4 = nn.Sequential(
      nn.Conv2d(64,128,3,1,1),
      nn.BatchNorm2d(128),
      nn.ReLU(),
      nn.MaxPool2d(2,2)
    )

    self.FC = nn.Sequential(
      nn.Linear(128,26)
    )


  def forward(self,x):
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    x = torch.flatten(x,1)
    x = self.FC(x)

    return x



In [14]:
net = NeuralNetwork()
net.load_state_dict(torch.load("/content/english.pt"))

<All keys matched successfully>

In [15]:
!pip install onnx==1.8.1

In [16]:
!pip install pytorch2keras

In [17]:
!pip install tensorflowjs

In [24]:
import torch
from pytorch2keras.converter import pytorch_to_keras

x = torch.randn(1, 1, 28, 28, requires_grad=False)
k_model = pytorch_to_keras(model=net,args=x,verbose=True,name_policy="shorts")
k_model.save('english_keras.h5')

INFO:pytorch2keras:Converter is called.
DEBUG:pytorch2keras:Input_names:
DEBUG:pytorch2keras:['input_0']
DEBUG:pytorch2keras:Output_names:
DEBUG:pytorch2keras:['output_0']
INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:None
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> input_0.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> output_0.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight FC.0.weight with shape (26, 128).
DEBUG:onnx2keras:Found weight FC.0.bias with shape (26,).
DEBUG:onnx2keras:Found weight 50 with shape (16, 1, 3, 3).
DEBUG:onnx2keras:Found weight 51 with shape (16,).
DEBUG:onnx2keras:Found weight 53 with shape (32, 16, 3, 3).
DEBUG:onnx2keras:Found weight 54 with shape (32,).
DEBUG:onnx2keras:Found weight 56 with shape (64, 32, 3, 3).
DEBUG:onnx2keras:Found weight 57 with shape (64,).
DEBUG:onnx2keras:Found weight 59 with shape (128, 64, 3, 3).
DEBUG:onnx2keras:Found weight 6

graph(%input_0 : Float(1, 1, 28, 28, strides=[784, 784, 28, 1], requires_grad=0, device=cpu),
      %FC.0.weight : Float(26, 128, strides=[128, 1], requires_grad=1, device=cpu),
      %FC.0.bias : Float(26, strides=[1], requires_grad=1, device=cpu),
      %50 : Float(16, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=0, device=cpu),
      %51 : Float(16, strides=[1], requires_grad=0, device=cpu),
      %53 : Float(32, 16, 3, 3, strides=[144, 9, 3, 1], requires_grad=0, device=cpu),
      %54 : Float(32, strides=[1], requires_grad=0, device=cpu),
      %56 : Float(64, 32, 3, 3, strides=[288, 9, 3, 1], requires_grad=0, device=cpu),
      %57 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %59 : Float(128, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %60 : Float(128, strides=[1], requires_grad=0, device=cpu)):
  %49 : Float(1, 16, 28, 28, strides=[12544, 784, 28, 1], requires_grad=1, device=cpu) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[3, 3]

DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:conv:Conv with bias
DEBUG:onnx2keras:conv:2D convolution
DEBUG:onnx2keras:conv:Paddings exist, add ZeroPadding layer
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 32, 14, 14), dtype=tf.float32, name=None), name='52/BiasAdd:0', description="created by layer '52'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Relu
DEBUG:onnx2keras:node_name: 37
DEBUG:onnx2keras:node_params: {'change_ordering': False, 'name_policy': 'shorts'}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 52).
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 32, 14, 14), dtype=tf.float32, name=None), name='37/Relu:0', description="created by layer '37'")
DEBUG:onnx2keras:

In [19]:
from tensorflow import keras

model = keras.models.load_model('/content/english_keras.h5')

In [20]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_0 (InputLayer)         [(None, 1, 28, 28)]       0         
_________________________________________________________________
49_pad (ZeroPadding2D)       (None, 1, 30, 30)         0         
_________________________________________________________________
49 (Conv2D)                  (None, 16, 28, 28)        160       
_________________________________________________________________
33 (Activation)              (None, 16, 28, 28)        0         
_________________________________________________________________
34 (MaxPooling2D)            (None, 16, 14, 14)        0         
_________________________________________________________________
52_pad (ZeroPadding2D)       (None, 16, 16, 16)        0         
_________________________________________________________________
52 (Conv2D)                  (None, 32, 14, 14)        4640

In [21]:
import tensorflowjs as tfjs

tfjs.converters.save_keras_model(model,"english_tfjs")